In [1]:
import warnings
warnings.filterwarnings('ignore')

import shutil
import pytorch_lightning as pl
import torch
import yaml
import os
import pickle
import logging
import sys
pkg_path = "/nfs/homedirs/nikoghos/project-1/"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)
    
from audio_classification.tools import *
import optuna
from pytorch_lightning import loggers as pl_loggers
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
import yaml
with open("../audio_classification/configs/crnn_urbansound8k.yaml", "r") as config_file:
    cfg = yaml.load(config_file)

In [3]:
cfg

{'MODEL': {'NAME': 'LitCRNN',
  'NUM_CLASSES': 10,
  'CRNN': {'MIXUP': True,
   'RANDOMISED_SMOOTHING': False,
   'INCLUDE_TOP': True,
   'INCLUDE_TRANSFORM': False}},
 'DATASET': {'NAME': 'UrbanSounds8K',
  'ANNOTATION_PATH': '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/metadata/UrbanSound8K.csv',
  'FILE_PATH': '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/audio/',
  'VAL_FOLD': 9,
  'AUGMENTATION': 'none',
  'NOISE_PATH': '/nfs/students/winter-term-2020/project-1/MUSAN/free-sound/',
  'WEIGHT': 'NORMAL'},
 'DATALOADER': {'BATCH_SIZE': 20, 'NUM_WORKERS': 20},
 'TRANSFORM': {'HOP_LENGTH': 256},
 'SOLVER': {'LOG_PATH': '../logs/crnn_us8k_fold9_smooth',
  'NUM_GPUS': 1,
  'MAX_EPOCH': 82,
  'MIN_EPOCH': 10,
  'LEARNING_RATE': 0.00017083989355164183,
  'WEIGHT_DECAY': 6.869889122096289e-09,
  'STEP_SIZE': 7,
  'GAMMA': 0.1,
  'ALPHA': 1.9369262397486933,
  'SIGMA': 1.0},
 'CHECKPOINT': {'SAVE_NAME': 'crnn-us8k_fold9_smooth',
  'SAVE_PATH': '../weights

In [4]:
from pytorch_lightning import Callback

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [5]:
def save_model(model, file_name, directory = "models"):
    model = model.cpu()
    model_dict = {cfg["MODEL"]["NAME"]:{"state_dict":model.state_dict(), "hparams": model.hparams}}
    if not os.path.exists(directory):
        os.makedirs(directory)
    pickle.dump(model_dict, open(os.path.join(directory, file_name), 'wb', 4))

In [8]:
def objective(trial):
    # Use this callback to collect the validation accuracies
        
    metrics_callback = MetricsCallback()
    
    trial_hparams = {"batch_size": 8, 
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
                     "weight_decay":  trial.suggest_loguniform("weight_decay", 1e-10, 1e-8),
                     "alpha": trial.suggest_loguniform("alpha", 0.1, 2.0)
                    }

    train_loader, val_loader, test_loader, class_weights = get_dataloader(cfg, trial_hparams, transform=get_transform(cfg), augment=cfg["DATASET"]["AUGMENTATION"])
    
    if class_weights is not None:
        class_weights = torch.tensor(class_weights).to(device=torch.device('cuda'))
    
    early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_acc")
    trainer = pl.Trainer(gpus=cfg["SOLVER"]["NUM_GPUS"],
                         min_epochs=cfg["SOLVER"]["MIN_EPOCH"],
                         max_epochs=37,
                         progress_bar_refresh_rate=10,
                         callbacks=[metrics_callback, early_stop_callback],
                         logger=True,
                        )
    
    model = get_model(cfg, class_weights, trial_hparams, train_loader, val_loader)
    
    model.prepare_data()
    trainer.fit(model)

    save_model(model, '{}.p'.format(trial.number), "checkpoints")

    # return validation accuracy from latest model, as that's what we want to maximize by our hyper param search
    return metrics_callback.metrics[-1]["val_acc"]

In [ ]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=20, timeout=86000)

[I 2021-02-06 21:16:55,958] A new study created in memory with name: no-name-0cf263c9-ed32-4f28-afd8-91ea56fec760


Transformed raw audio into melspectrogram in dataloader.
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | transform      | Sequential  | 0     
5  | conv1          | Conv2d      | 640   
6  | bn1            | BatchNorm2d | 128   
7  | dropout1       | Dropout     | 0     
8  | conv2          | Conv2d      | 73.9 K
9  | bn2            | BatchNorm2d | 256   
10 | dropout2       | Dropout     | 0     
11 | conv3          | Conv2d      | 147 K 
12 | bn3            | BatchNorm2d | 256   
13 | dropout3       | Dropout     | 0     
14 | conv4          | Conv2d      | 147 K 
15 | bn4            | BatchNorm2d | 256   
16 | dropout4       | Dropout     | 0     
17 | rnn            | GRU         |

[I 2021-02-06 22:27:57,091] Trial 0 finished with value: 0.6672953963279724 and parameters: {'learning_rate': 0.0006553230752940021, 'weight_decay': 6.438000458474734e-09, 'alpha': 1.2164076566193807}. Best is trial 0 with value: 0.6672953963279724.



Transformed raw audio into melspectrogram in dataloader.
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none


...


Transformed raw audio into melspectrogram in dataloader.
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | transform      | Sequential  | 0     
5  | conv1          | Conv2d      | 640   
6  | bn1            | BatchNorm2d | 128   
7  | dropout1       | Dropout     | 0     
8  | conv2          | Conv2d      | 73.9 K
9  | bn2            | BatchNorm2d | 256   
10 | dropout2       | Dropout     | 0     
11 | conv3          | Conv2d      | 147 K 
12 | bn3            | BatchNorm2d | 256   
13 | dropout3       | Dropout     | 0     
14 | conv4          | Conv2d      | 147 K 
15 | bn4            | BatchNorm2d | 256   
16 | dropout4       | Dropout     | 0     
17 | rnn            | GRU         |

[I 2021-02-07 15:05:18,967] Trial 14 finished with value: 0.613476037979126 and parameters: {'learning_rate': 3.0041830755099535e-05, 'weight_decay': 1.1231592979337692e-10, 'alpha': 0.47286031932758876}. Best is trial 2 with value: 0.6982245445251465.



Transformed raw audio into melspectrogram in dataloader.
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none
Data augmentation for US8K dataset: none


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name           | Type        | Params
------------------------------------------------
0  | val_precision  | Precision   | 0     
1  | val_recall     | Recall      | 0     
2  | test_precision | Precision   | 0     
3  | test_recall    | Recall      | 0     
4  | transform      | Sequential  | 0     
5  | conv1          | Conv2d      | 640   
6  | bn1            | BatchNorm2d | 128   
7  | dropout1       | Dropout     | 0     
8  | conv2          | Conv2d      | 73.9 K
9  | bn2            | BatchNorm2d | 256   
10 | dropout2       | Dropout     | 0     
11 | conv3          | Conv2d      | 147 K 
12 | bn3            | BatchNorm2d | 256   
13 | dropout3       | Dropout     | 0     
14 | conv4          | Conv2d      | 147 K 
15 | bn4            | BatchNorm2d | 256   
16 | dropout4       | Dropout     | 0     
17 | rnn            | GRU         |

In [8]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 17
Best trial:
  Value: 0.6708158850669861
  Params: 
    learning_rate: 9.521089031949996e-06
    weight_decay: 1.131463918804116e-06
    alpha: 2.3485658333624597


In [8]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
best_trial = study.best_trial

print("  Value: {}".format(best_trial.value))

print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 50
Best trial:
  Value: 0.9744158387184143
  Params: 
    learning_rate: 0.00017438033988242943
    weight_decay: 6.944923420383138e-05
    alpha: 1.7231602078617747


In [8]:
shutil.rmtree("checkpoints")

In [1]:
print("Number of finished trials: 50")

print("Best trial:")

print("0.73614310462657")

print("  Params: ")
print("  learning_rate:  0.0004881731392294949")
print("  weight_decay: 2.628328526113425e-08")
print("  alpha: 0.7608399210841633")


Number of finished trials: 50
Best trial:
0.73614310462657
  Params: 
  learning_rate:  0.0004881731392294949
  weight_decay: 2.628328526113425e-08
  alpha: 0.7608399210841633
